Now that we know which models are performing better, it's time to perform cross validation and tune hyperparameters.
- Do a google search for hyperparameter ranges for each type of model.

GridSearch/RandomSearch are a great methods for checking off both of these tasks.
- BUT we have a problem - if we calculated a numerical value to encode city (such as the mean of sale prices in that city) on the training data, we can't cross validate 
- The rows in each validation fold were part of the original calculation of the mean for that city - that means we're leaking information!
- While sklearn's built in functions are extremely useful, sometimes it is necessary to do things ourselves

You need to create two functions to replicate what Gridsearch does under the hood

**`custom_cross_validation()`**
- Should take the training data, and divide it into multiple train/validation splits. 
- Look into `sklearn.model_selection.KFold` to accomplish this - the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) shows how to split a dataframe and loop through the indexes of your split data. 
- Within your function, you should compute the city means on the training folds just like you did in Notebook 1 - you may have to re-join the city column to do this - and then join these values to the validation fold

**`hyperparameter_search()`**
- Should take the validation and training splits from your previous function, along with your dictionary of hyperparameter values
- For each set of hyperparameter values, fit your chosen model on each set of training folds, and take the average of your chosen scoring metric. [itertools.product()](https://docs.python.org/3/library/itertools.html) will be helpful for looping through all combinations of hyperparameter values
- Your function should output the hyperparameter values corresponding the highest average score across all folds. Alternatively, it could also output a model object fit on the full training dataset with these parameters.

Docstrings have been provided below to get you started. Once you're done developing your functions, you should move them to `functions_variables.py` to keep your notebook clean 

Bear in mind that these instructions are just one way to tackle this problem - the inputs and output formats don't need to be exactly as specified here.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from itertools import product
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

X_train = pd.read_csv('../data/training/X_train.csv')
y_train = pd.read_csv('../data/training/y_train.csv')
X_test = pd.read_csv('../data/testing/X_test.csv')
y_test = pd.read_csv('../data/testing/y_test.csv')
df_final = pd.read_csv('../data/processed/df_final_processed_no_bins.csv')

In [2]:
def custom_cross_validation(df, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    for train_index, val_index in kf.split(df):
        train_df = df.iloc[train_index]
        val_df = df.iloc[val_index]

        # Compute city means on the training data
        city_means = train_df.groupby(
            'city')['price'].mean().rename('city_mean')

        # Merge the city means with the training and validation data
        train_df = train_df.join(city_means, on='city')
        val_df = val_df.join(city_means, on='city', rsuffix='_val')

        yield train_df, val_df

In [3]:
def hyperparameter_search(model_class, param_grid, df, n_splits=5):
    best_score = float('inf')
    best_params = None
    best_model = None

    param_combinations = list(product(*param_grid.values()))

    for params in param_combinations:
        param_dict = dict(zip(param_grid.keys(), params))
        model = model_class(**param_dict)

        scores = []
        for train_df, val_df in custom_cross_validation(df, n_splits):
            X_train = train_df.drop(['price'], axis=1)
            y_train = train_df['price']
            X_val = val_df.drop(['price'], axis=1)
            y_val = val_df['price']

            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
            score = mean_squared_error(y_val, y_pred)
            scores.append(score)

        avg_score = np.mean(scores)

        if avg_score < best_score:
            best_score = avg_score
            best_params = param_dict
            best_model = model_class(**best_params)
            best_model.fit(df.drop(['price'], axis=1), df['price'])

    return best_params, best_model, best_score

In [4]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Define the categorical and numerical features
categorical_features = ['type', 'city', 'is_foreclosure', ]
numerical_features = [
    'year_built', 'sqft', 'baths', 'stories', 'beds', 'garage',  'basement', 'garage_1_or_more', 'garage_2_or_more',
    'hardwood_floors', 'laundry_room', 'recreation_facilities', 'shopping',
    'single_story', 'two_or_more_stories'
]

# Convert boolean features to integers if necessary
if df_final['is_foreclosure'].dtype == 'bool':
    df_final['is_foreclosure'] = df_final['is_foreclosure'].astype(int)

# Define preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Define preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the model
model = xgb.XGBRegressor()

# Create and combine preprocessing and modeling pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'model__n_estimators': [50, 100, 200],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__max_depth': [3, 5, 7],
    'model__min_child_weight': [1, 3, 5],
    'model__gamma': [0, 0.1, 0.2],
    'model__subsample': [0.7, 0.8, 0.9],
    'model__colsample_bytree': [0.7, 0.8, 0.9],
    'model__reg_alpha': [0, 0.1, 1],
    'model__reg_lambda': [0.1, 1, 10]
}

# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5,
                           scoring='neg_mean_squared_error', verbose=2)

# Fit the model
grid_search.fit(X_train, y_train)

# Output the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 5 folds for each of 19683 candidates, totalling 98415 fits
[CV] END model__colsample_bytree=0.7, model__gamma=0, model__learning_rate=0.01, model__max_depth=3, model__min_child_weight=1, model__n_estimators=50, model__reg_alpha=0, model__reg_lambda=0.1, model__subsample=0.7; total time=   0.0s
[CV] END model__colsample_bytree=0.7, model__gamma=0, model__learning_rate=0.01, model__max_depth=3, model__min_child_weight=1, model__n_estimators=50, model__reg_alpha=0, model__reg_lambda=0.1, model__subsample=0.7; total time=   0.0s
[CV] END model__colsample_bytree=0.7, model__gamma=0, model__learning_rate=0.01, model__max_depth=3, model__min_child_weight=1, model__n_estimators=50, model__reg_alpha=0, model__reg_lambda=0.1, model__subsample=0.7; total time=   0.0s
[CV] END model__colsample_bytree=0.7, model__gamma=0, model__learning_rate=0.01, model__max_depth=3, model__min_child_weight=1, model__n_estimators=50, model__reg_alpha=0, model__reg_lambda=0.1, model__subsample=0.7; total ti

KeyboardInterrupt: 

In [ ]:
# develop your custom functions here

def custom_cross_validation(training_data, n_splits =5):
    '''creates n_splits sets of training and validation folds

    Args:
      training_data: the dataframe of features and target to be divided into folds
      n_splits: the number of sets of folds to be created

    Returns:
      A tuple of lists, where the first index is a list of the training folds, 
      and the second the corresponding validation fold

    Example:
        >>> output = custom_cross_validation(train_df, n_splits = 10)
        >>> output[0][0] # The first training fold
        >>> output[1][0] # The first validation fold
        >>> output[0][1] # The second training fold
        >>> output[1][1] # The second validation fold... etc.
    '''

    return training_folds, validation_folds

def hyperparameter_search(training_folds, validation_folds, param_grid):
    '''outputs the best combination of hyperparameter settings in the param grid, 
    given the training and validation folds

    Args:
      training_folds: the list of training fold dataframes
      validation_folds: the list of validation fold dataframes
      param_grid: the dictionary of possible hyperparameter values for the chosen model

    Returns:
      A list of the best hyperparameter settings based on the chosen metric

    Example:
        >>> param_grid = {
          'max_depth': [None, 10, 20, 30],
          'min_samples_split': [2, 5, 10],
          'min_samples_leaf': [1, 2, 4],
          'max_features': ['sqrt', 'log2']} # for random forest
        >>> hyperparameter_search(output[0], output[1], param_grid = param_grid) 
        # assuming 'ouput' is the output of custom_cross_validation()
        [20, 5, 2, 'log2'] # hyperparams in order
    '''

    return hyperparameters


In [ ]:
# perform tuning and cross validation here

We want to make sure that we save our models.  In the old days, one just simply pickled (serialized) the model.  Now, however, certain model types have their own save format.  If the model is from sklearn, it can be pickled, if it's xgboost, for example, the newest format to save it in is JSON, but it can also be pickled.  It's a good idea to stay with the most current methods. 
- you may want to create a new `models/` subdirectory in your repo to stay organized

In [ ]:
# save your best model here

Once you've identified which model works the best, implement a prediction pipeline to make sure that you haven't leaked any data, and that the model could be easily deployed if desired.
- Your pipeline should load the data, process it, load your saved tuned model, and output a set of predictions
- Assume that the new data is in the same JSON format as your original data - you can use your original data to check that the pipeline works correctly
- Beware that a pipeline can only handle functions with fit and transform methods.
- Classes can be used to get around this, but now sklearn has a wrapper for user defined functions.
- You can develop your functions or classes in the notebook here, but once they are working, you should import them from `functions_variables.py` 

In [ ]:
# Build pipeline here

Pipelines come from sklearn.  When a pipeline is pickled, all of the information in the pipeline is stored with it.  For example, if we were deploying a model, and we had fit a scaler on the training data, we would want the same, already fitted scaling object to transform the new data with.  This is all stored when the pipeline is pickled.
- save your final pipeline in your `models/` folder

In [ ]:
# save your pipeline here